# Upload clips to Zooniverse
Script to generate short clips from underwater movies and upload them to Zooniverse.

## Initial requirements
Import required packages, indicate how many clips to generate, the folder to store the new clips, and relevant information to upload the clips to Zooniverse.
Note, we use the [Python SDK for Panoptes](https://github.com/zooniverse/panoptes-python-client) to upload clips to Zooniverse. 
Only Zooniverse project admins can upload clips to the Koster lab Zooniverse project.

In [ ]:
# Import required packages
import os
import subprocess
import math
import csv
from datetime import date
from panoptes_client import SubjectSet, Subject, Project, Panoptes #needed to upload clips to Zooniverse

# Specify how many clips to generate and its length (seconds)
n_clips = 1000
clip_length = 10

# Specify the folder location to store the clips
while True:
    location = input('Enter the full path for the image directory, or enter "." '
                     'to use the current directory' + '\clips')
    if location == '.':
        location = os.getcwd()
        break
    else:
        if os.path.exists(location):
            break
        else:
            print('That entry is not a valid path for an existing directory')
            retry = input('Enter "y" to try again, any other key to exit' + '\n')
            if retry.lower() != 'y':
                quit()

# Connect to Zooniverse with your username and password
Panoptes.connect(username='', password='')

# Specify the project number of the koster lab
koster_project = Project(9747)

## Create a list of clips to generate

In [ ]:
# Query the koster lab database to randomly select n clips from movie sections that haven't been clipped yet
clip_list


## Generate the clips

Use the list generated in the previous step to select movie files and split them into small sections of video (clips). Update the Koster lab database and subject_metadata when the clip has succesfully been generated.

In [ ]:
#Create empty subject metadata to keep track of the clips generated
subject_metadata = {}

# Generate one clip at the time, update the koster lab database and the subject_metadata
for clip in clip_list:
    # Generate and store the clip
    subject_filename = str(video_filename) + "_" + str(int(clip_start)) + ".mp4"
    fileoutput = location + os.sep + subject_filename
    subprocess.call(["ffmpeg", "-ss", str(clip_start), "-t", clip_duration, "-i", video_filename, "-c", "copy", "-force_key_frames", "1", fileoutput])
    
     # Add clip information to the koster lab database
    clip_id = 
    filename = subject_filename
    start_time = clip_start
    end_time = clip_start + clip_duration
    clip_date = date.today().strftime("%d_%m_%Y")
    
    # Add clip information to the subject_metadata
    subject_metadata[clip] = {'filename': subject_filename, '#start_time': start_time, '#end_time': end_time}

print(len(clip_list), ' clips have been generated in ', location, '.')

## Create Zooniverse dataset to upload the clips

In [ ]:
# Create a new subject set (the Zooniverse dataset that will store the clips)
set_name = input('clips_'+ date.today().strftime("%d_%m_%Y"))
previous_subjects = []

try:
    # check if the subject set already exits
    subject_set = SubjectSet.where(project_id=koster_project.id, display_name=set_name).next()
    print('You have chosen to upload ', len(subject_metadata), ' files to an existing subject set',  set_name)
    retry = input('Enter "n" to cancel this upload, any other key to continue' + '\n')
    if retry.lower() == 'n':
        quit()
    for subject in subject_set.subjects:
        previous_subjects.append(subject.metadata['filename'])
except StopIteration:
    print('You have chosen to upload ', len(subject_metadata), ' files to an new subject set ',  set_name)
    retry = input('Enter "n" to cancel this upload, any other key to continue' + '\n')
    if retry.lower() == 'n':
        quit()
    # create a new subject set for the new data and link it to the project above
    subject_set = SubjectSet()
    subject_set.links.project = koster_project
    subject_set.display_name = set_name
    subject_set.save()

## Upload clips to Zooniverse

In [ ]:
# Upload the clips to the project
print('Uploading subjects, this could take a while!')
new_subjects = 0

for filename, metadata in subject_metadata.items():
    try:
        if filename not in previous_subjects:
            subject = Subject()
            subject.links.project = koster_project
            subject.add_location(location + os.sep + filename)
            subject.metadata.update(metadata)
            subject.save()
            subject_set.add(subject.id)
            print(filename)
            new_subjects += 1
    except panoptes_client.panoptes.PanoptesAPIException:
        print('An error occurred during the upload of ', filename)
print(new_subjects, 'new subjects created and uploaded')

#Generate a csv file with all the uploaded clips
uploaded = 0
with open(location + os.sep + 'Uploaded subjects.csv', 'wt') as file:
    subject_set = SubjectSet.where(project_id=koster_project.id, display_name=set_name).next()
    for subject in subject_set.subjects:
        uploaded += 1
        file.write(subject.id + ',' + list(subject.metadata.values())[0] + '\n')
    print(uploaded, ' subjects found in the subject set, see the full list in Uploaded subjects.csv.')